# IPO Random Forest Regression
Predicting Excess Returns using pre-IPO Data and Random Forest Regressions

In [344]:
# Library imports
import psycopg2
import os
import pandas as pd
import numpy as np
import requests

## Gathering data

In [345]:
# Establish connection to PostgreSQL
conn = psycopg2.connect(os.environ.get('DB_CONNECTION_STRING'))

In [346]:
# pre-IPO data
query = 'SELECT DISTINCT ON ("companyName") * FROM ipos ORDER BY "companyName", "createdAt" DESC NULLS LAST;'
ipos = pd.read_sql(query, conn)
ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   id                      188 non-null    int64              
 1   symbol                  188 non-null    object             
 2   companyName             188 non-null    object             
 3   expectedDate            188 non-null    datetime64[ns, UTC]
 4   auditor                 188 non-null    object             
 5   market                  188 non-null    object             
 6   cik                     188 non-null    object             
 7   address                 188 non-null    object             
 8   city                    188 non-null    object             
 9   state                   188 non-null    object             
 10  zip                     188 non-null    object             
 11  phone                   188 non-null    objec

In [347]:
# Company data
query = f'SELECT * FROM companies ORDER BY "companyName";'
companies = pd.read_sql(query, conn)
companies.iloc[0]

id                                                              369
symbol                                                          TXG
companyName                                      10X Genomics, Inc.
exchange                                                     NASDAQ
industry                                              Biotechnology
website                                  http://www.10xgenomics.com
description       10X Genomics, Inc. is a life science technolog...
CEO                                                   Serge Saxonov
securityName                               10x Genomics Inc Class A
issueType                                                        cs
sector                                            Health Technology
primarySicCode                                                 2836
employees                                                       584
address                                   6230 Stoneridge Mall Road
address2                                        

In [348]:
# Company price data
companies_ids = tuple(companies['id'].to_numpy())
query = f'SELECT * FROM prices WHERE "companyId" IN {companies_ids} ORDER BY "companyId", "date";'
prices = pd.read_sql(query, conn)
prices['date'] = pd.to_datetime(prices['date']) 

In [349]:
# Benchmark price data
r = requests.get(os.environ.get('BENCHMARK_QUERY_STRING'))
raw_benchmark_data = r.json()

In [350]:
benchmark = pd.DataFrame.from_dict(raw_benchmark_data)
benchmark['date'] = pd.to_datetime(benchmark['date']) 

## Build Labeled Series

In [351]:
# 1. Create 21-trading day return series for SPY (average # of trading days per month)
# 2. Extract return for each IPO and its excess return to SPY over the same period
#    this should be a function that takes a trading day size and benchmark return series
# 3. Combine returns with pre-IPO data
# 4. Remove companies that do not have return data available given trading window size
# 5. Convert categorical data to one-hot or binary encoding
# 6. Finialze sample data (X, Y)

In [352]:
# Create a period return series
def period_return(df, period, price_column, return_type='discrete', zero_idx=True):
    '''
    Create a period return series from DataFrame (requires ASC order).
    '''
    
    types = ('discrete', 'continuous')
    if return_type not in types:
        raise TypeError(f'return_type needs to be of type: {types}')

    zero_period = period
    if zero_idx == True:
        zero_period = period - 1

    if return_type == 'discrete':
        return (df[price_column][zero_period:] - df[price_column][:-zero_period].values) / df[price_column][:-zero_period].values
    else:
        return np.log(df[price_column][zero_period:]) - np.log(df[price_column][:-zero_period].values)

In [353]:
# 1. Benchmark Returns
benchmark['returns21d'] = period_return(benchmark, 21, 'close')

In [354]:
# 2. IPO returns
def add_returns_groupby(df, groupby, price_column, period, returns_column='returns'):
    '''
    Adds a return column to original DataFrame. Calculations are done arounding to groupby key.
    
    :returns: Original DataFrame with new column.
    '''
    grouped = df.groupby(groupby)
    returns_series = []
    first_series = None
    for _, group in grouped:
        if first_series is None:
            first_series = period_return(group, period, price_column).rename(returns_column)
        else:
            returns_series.append(period_return(group, period, price_column).rename(returns_column))

    return df.join(first_series.append(returns_series))

In [355]:
# 2. IPO returns
prices = add_returns_groupby(prices, 'companyId', 'close', 21)

In [356]:
# Excess returns
prices = prices.merge(benchmark[['date', 'returns21d']])
prices['ex_returns'] = prices['returns'] - prices['returns21d']

In [357]:
# Match sybmol to id
prices = prices.merge(companies[['id', 'symbol']], left_on='companyId', right_on='id')
prices

,id_x,date,high,low,volume,open,close,uHigh,uLow,uVolume,...,change,changePercent,createdAt,updatedAt,companyId,returns,returns21d,ex_returns,id_y,symbol
0,20936,2019-08-01,13.22,7.66,9314399.0,13.01,8.48,13.22,7.66,9314399.0,...,0.00,0.0000,2020-02-06 23:34:00.006000+00:00,2020-02-06 23:34:00.006000+00:00,332,NaN,0.168240,NaN,332,SNDL
1,20937,2019-08-02,10.48,8.42,2693863.0,8.42,10.45,10.48,8.42,2693863.0,...,1.97,23.2311,2020-02-06 23:34:00.006000+00:00,2020-02-06 23:34:00.006000+00:00,332,NaN,0.182069,NaN,332,SNDL
2,20938,2019-08-05,11.82,10.47,2206717.0,10.69,11.70,11.82,10.47,2206717.0,...,1.25,11.9617,2020-02-06 23:34:00.006000+00:00,2020-02-06 23:34:00.006000+00:00,332,NaN,0.107545,NaN,332,SNDL
3,20939,2019-08-06,13.21,11.99,2180774.0,12.00,13.00,13.21,11.99,2180774.0,...,1.30,11.1111,2020-02-06 23:34:00.006000+00:00,2020-02-06 23:34:00.006000+00:00,332,NaN,0.097477,NaN,332,SNDL
4,20940,2019-08-07,13.22,12.20,1611203.0,13.05,12.85,13.22,12.20,1611203.0,...,-0.15,-1.1538,2020-02-06 23:34:00.006000+00:00,2020-02-06 23:34:00.006000+00:00,332,NaN,0.113691,NaN,332,SNDL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16076,31903,2020-07-06,96.51,73.11,13467931.0,73.39,81.19,96.51,73.11,13467931.0,...,11.78,16.9700,2020-07-07 09:30:18.182000+00:00,2020-07-07 09:30:18.182000+00:00,548,NaN,-0.072555,NaN,548,LMND
16077,32029,2020-07-07,89.38,75.00,4602812.0,83.80,78.79,89.38,75.00,4602812.0,...,-2.40,-2.9600,2020-07-08 09:30:11.720000+00:00,2020-07-08 09:30:11.720000+00:00,548,NaN,-0.099618,NaN,548,LMND
16078,32155,2020-07-08,79.39,68.06,3499175.0,79.00,68.51,79.39,68.06,3499175.0,...,-10.28,-13.0500,2020-07-09 09:30:12.058000+00:00,2020-07-09 09:30:12.058000+00:00,548,NaN,-0.014198,NaN,548,LMND
16079,32282,2020-07-09,79.91,69.03,4178671.0,73.97,77.01,79.91,69.03,4178671.0,...,8.50,12.4100,2020-07-10 09:30:14.207000+00:00,2020-07-10 09:30:14.207000+00:00,548,NaN,0.016505,NaN,548,LMND


In [358]:
# 3/4. Combine the first return observation with pre-IPO data
def combine_return_data(prices, ipos, return_col):
    '''
    Finds a single return to match to the pre-ipo data
    '''
    grouped = prices.groupby('symbol')
    returns_series = []
    for symbol, group in grouped:
        temp_return = group[~pd.isna(group[return_col])][return_col]
        if len(temp_return) > 0:
            returns_series.append({
                'symbol': symbol,
                'ex_returns': temp_return.iloc[0]
            })
    return ipos.merge(pd.DataFrame.from_dict(returns_series), on='symbol')

In [359]:
ipos = combine_return_data(prices, ipos, 'ex_returns')

## Input Features - Pre-IPO Data Variables
#### Categorical
1. market
2. state
3. industry (v2)
4. sector (v2)

#### Numeric
1. employees
2. sharesOffered
3. priceLow
4. totalExpenses
5. shareholderShares
6. sharesOutstanding
7. revenue
8. netIncome
9. totalAssets
10. totalLiabilities
11. stockholderEquity
12. amount
13. percentOffered

In [361]:
def create_labeled_data(data, categorical, numeric):
    print('test')
    
print(pd.get_dummies(ipos['state']))

        CA  CO  CT  DC  DE  FL  GA  IL  KS  ...  MD  MO  NC  NJ  NY  PA  TN  \
0    0   1   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
1    0   1   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
2    1   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
3    0   1   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
4    1   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..   
105  1   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
106  1   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
107  1   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
108  0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   1   0   0   
109  0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   

     TX  UT  WA  
0     0   0   0  
1     0   0   0